In [ ]:
import tensorflow as tf
from tensorflow import keras
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [ ]:
def vgg11(outputs):
    net = keras.models.Sequential()
    filters = 64
    for n_conv in [1, 1, 2, 2, 2]:
        for _ in range(n_conv):
            net.add(
                keras.layers.Conv2D(filters, 3, padding="same", activation="relu")
            )
        net.add(
            keras.layers.MaxPool2D(2, 2)
        )
        filters = filters << 1
    net.add(keras.layers.Flatten())
    net.add(
        keras.layers.Dense(512, activation='relu')
    )
    net.add(
        keras.layers.Dense(256, activation='relu')
    )
    net.add(
        keras.layers.Dense(outputs)
    )
    return net

In [ ]:
net = vgg11(2)

In [ ]:
def load_data(resize=(256, 256), batch_size=50):
    import os 
    def __load_data(path):
        sub_paths = os.listdir(path)
        datas = []
        labels = []
        for label, label_name in enumerate(sub_paths):
            for simple in os.listdir(path + os.sep + label_name):
                if simple.endswith('jpg'):
                    labels.append(label)
                    datas.append(path + os.sep + label_name + os.sep + simple)
        datas = tf.data.Dataset.from_tensor_slices(datas)
        labels = tf.data.Dataset.from_tensor_slices(labels)
        def map_img(img_path):
            return tf.image.resize(tf.image.decode_jpeg(tf.io.read_file(img_path)), resize) / 255.
        datas = datas.map(map_img, num_parallel_calls=tf.data.AUTOTUNE)
        ds = tf.data.Dataset.zip((datas, labels))
        # return ds
        return ds.shuffle(buffer_size=len(datas), reshuffle_each_iteration=True).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        # return datas.batch(batch_size).prefetch(tf.data.AUTOTUNE), labels.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return __load_data('./training_set/'), __load_data('./test_set/')

In [ ]:
# 加载数据
BATCH_SIZE = 50
train_iter, test_iter = load_data(batch_size=BATCH_SIZE, resize=(200, 200))

In [ ]:

ETA = 1e-4
EPOCHS = 20

optimizer = keras.optimizers.SGD(learning_rate=ETA)
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
accuracy = keras.metrics.SparseCategoricalAccuracy()

def acc_fn(y, y_hat):
    accuracy.reset_state()
    accuracy.update_state(y, y_hat)
    return accuracy.result().numpy()

for epoch in range(EPOCHS):
    for valid_x, valid_y in train_iter.take(1):
        pass
    for step, (x, y) in enumerate(train_iter):
        with tf.GradientTape() as gt:
            y_hat = net(x)
            y_hat = tf.nn.softmax(y_hat)
            l = loss_fn(y, y_hat)
        grads = gt.gradient(l, net.trainable_weights)
        optimizer.apply_gradients(zip(grads, net.trainable_weights))
        if step % 10 == 0:
            acc = acc_fn(y, y_hat)
            valid_acc = acc_fn(valid_y, net(valid_x))
            print("loss: %.6f, acc: %.4f, valid acc: %.4f" % (l, acc, valid_acc))
            